In [1]:
import pandas as pd
import re
from collections import namedtuple
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [2]:
pop_2010_2019 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/GA/pop_est_2010-2019_ga.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Appling,18370,18472,18423,18428,18438,18526,18386
1,Atkinson,8264,8215,8340,8244,8182,8262,8165
2,Bacon,11173,11178,11231,11275,11205,11132,11164
3,Baker,3348,3288,3194,3183,3159,3086,3038
4,Baldwin,46178,45910,45549,45259,44963,44893,44890


In [3]:
pop_2020_2021 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/GA/pop_est_2020-2021_ga.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Appling,18428,18488
1,Atkinson,8301,8391
2,Bacon,11123,11079
3,Baker,2844,2819
4,Baldwin,43773,43781


In [4]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined.drop(index=pop_combined.index[-1],axis=0,inplace=True)
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Appling,18370,18472,18423,18428,18438,18526,18386,18428,18488
1,Atkinson,8264,8215,8340,8244,8182,8262,8165,8301,8391
2,Bacon,11173,11178,11231,11275,11205,11132,11164,11123,11079
3,Baker,3348,3288,3194,3183,3159,3086,3038,2844,2819
4,Baldwin,46178,45910,45549,45259,44963,44893,44890,43773,43781


In [5]:
metros = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_ga = metros_df[metros_df['State Name']=='Georgia']
metros_ga.head()

,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
12,"Douglas, GA",Micropolitan Statistical Area,Atkinson,Georgia
15,"Milledgeville, GA",Micropolitan Statistical Area,Baldwin,Georgia
17,"Atlanta-Sandy Springs-Alpharetta, GA",Metropolitan Statistical Area,Barrow,Georgia
18,"Atlanta-Sandy Springs-Alpharetta, GA",Metropolitan Statistical Area,Bartow,Georgia
25,"Fitzgerald, GA",Micropolitan Statistical Area,Ben Hill,Georgia


In [6]:
file ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/GA/GA_2013.csv'
ga = pd.read_csv(file)
df_ga = pd.DataFrame(ga)
df_ga.head(10)

,Date and Time,Area: County,# of Fatalities (Person Table),# Serious Injuries,# Visible Injuries,# Complaint Injuries
0,01/03/2013 12:02 PM,Crisp,0.0,0.0,0.0,0.0
1,01/11/2013 07:49 AM,Chatham,0.0,0.0,0.0,0.0
2,07/27/2013 06:23 PM,Newton,0.0,0.0,0.0,0.0
3,04/23/2013 10:52 AM,NaN,0.0,0.0,0.0,0.0
4,10/01/2013 08:43 AM,Troup,0.0,0.0,0.0,0.0
5,12/30/2013 01:29 PM,Liberty,0.0,0.0,1.0,2.0
6,12/28/2013 10:59 PM,Paulding,0.0,0.0,0.0,1.0
7,11/30/2013 12:00 PM,Gwinnett,0.0,0.0,0.0,7.0
8,12/09/2013 10:38 PM,NaN,0.0,0.0,0.0,0.0
9,12/16/2013 04:52 PM,Jackson,0.0,0.0,0.0,0.0


In [7]:
df_ga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327263 entries, 0 to 327262
Data columns (total 6 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Date and Time                   327263 non-null  object 
 1   Area: County                    312821 non-null  object 
 2   # of Fatalities (Person Table)  326235 non-null  float64
 3   # Serious Injuries              326235 non-null  float64
 4   # Visible Injuries              326235 non-null  float64
 5   # Complaint Injuries            326235 non-null  float64
dtypes: float64(4), object(2)
memory usage: 15.0+ MB


In [8]:
df_ga_new = df_ga.groupby(['Area: County']).agg({'# of Fatalities (Person Table)':'sum','# Serious Injuries':'sum','# Visible Injuries':'sum','# Complaint Injuries':'sum'}).reset_index()
df_ga_new['total_crashes'] = df_ga_new.sum(axis = 1)
df_ga_new['Injuries'] = df_ga_new['# Serious Injuries'] + df_ga_new['# Visible Injuries'] + df_ga_new['# Complaint Injuries']
df_ga_new = df_ga_new.drop(columns=['# Serious Injuries','# Visible Injuries','# Complaint Injuries'],axis=1)
df_ga_new.columns = ['County','fatalaties','total_crashes','injuries']
df_ga_new.head()

C:\Users\iulia\AppData\Local\Temp/ipykernel_20556/416597001.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_ga_new['total_crashes'] = df_ga_new.sum(axis = 1)


,County,fatalaties,total_crashes,injuries
0,Appling,1.0,182.0,181.0
1,Atkinson,0.0,47.0,47.0
2,Bacon,2.0,111.0,109.0
3,Baker,0.0,17.0,17.0
4,Baldwin,6.0,640.0,634.0


In [9]:
for filename in os.listdir("C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/GA/"):
    if filename.startswith("GA_"):
        year = re.findall(r'\d{4}',filename)
        for var in year:
            name = int(var)
        ga_file = pd.read_csv(os.path.join("C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/GA/", filename))
        df_ga = pd.DataFrame(ga_file)
        df_ga_new = df_ga.groupby(['Area: County']).agg({'# of Fatalities (Person Table)':'sum','# Serious Injuries':'sum','# Visible Injuries':'sum','# Complaint Injuries':'sum'}).reset_index()
        df_ga_new['total_crashes'] = df_ga_new.sum(axis = 1)
        df_ga_new['Injuries'] = df_ga_new['# Serious Injuries'] + df_ga_new['# Visible Injuries'] + df_ga_new['# Complaint Injuries']
        df_ga_new = df_ga_new.drop(columns=['# Serious Injuries','# Visible Injuries','# Complaint Injuries'],axis=1)
        df_ga_new.columns = ['County','fatalaties','total_crashes','injuries']
        df_ga_new = df_ga_new[['County','fatalaties','injuries','total_crashes']]
        df_combined_ga = pd.merge(df_ga_new,pop_combined[['County', name]],on="County",how='left')
        df_combined_ga_new = pd.merge(df_combined_ga,metros_ga[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
        df_combined_ga_new.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/%s_GA.xlsx' % name, engine='xlsxwriter',index=False)
    else:
        continue

C:\Users\iulia\AppData\Local\Temp/ipykernel_20556/2111567298.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_ga_new['total_crashes'] = df_ga_new.sum(axis = 1)
